In [1]:
import pandas as pd
import pickle 
from scipy.stats import poisson
from numpy import sqrt
import math

In [2]:
dict_table = pickle.load(open('dict_table.pickle','rb'))
historical_data = pd.read_csv('Clean_fifa_worldcup_matches.csv')
fixture = pd.read_csv('Clean_fifa_worldcup_fixture.csv')

In [3]:
home = historical_data[['HomeTeam','HomeGoals','AwayGoals']]
away = historical_data[['AwayTeam','HomeGoals','AwayGoals']]
home = home.rename(columns={'HomeTeam':'Team','HomeGoals':'ScoredGoals','AwayGoals':'ConcededGoals'})
away = away.rename(columns={'AwayTeam':'Team','HomeGoals':'ConcededGoals','AwayGoals':'ScoredGoals'})
#team_strength = pd.concat([home,away],ignore_index=True).groupby('Team').mean()

In [4]:
Actuality = ((1.1-(2022 - historical_data['Year'])/92)**4)*3
Actuality2 = pd.concat([Actuality,Actuality],ignore_index=True)
WeightedGoals = pd.concat([home,away],ignore_index=True)
WeightedGoals['ScoredGoals'] = WeightedGoals['ScoredGoals'] * Actuality2
WeightedGoals['ConcededGoals'] = WeightedGoals['ConcededGoals'] * Actuality2
team_strength = WeightedGoals.groupby('Team').mean()

In [5]:
def predict_points(Home,Away):
    if Home  in team_strength.index and Away in team_strength.index:
        lamb_home = (team_strength.at[Home,'ScoredGoals']*team_strength.at[Away,'ConcededGoals'])
        lamb_away = (team_strength.at[Away,'ScoredGoals']*team_strength.at[Home,'ConcededGoals'])
        prob_home, prob_away   , prob_draw = 0,0,0
        for x in range(0,10):
            for y in range(0,10):
                p = poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x == y:
                    prob_draw += p
                elif x>y:
                    prob_home += p
                elif y>x:
                    prob_away += p
        points_home = 3*prob_home + prob_draw
        point_away = 3*prob_away + prob_draw
        return(points_home,point_away)
        #return(prob_home*100,prob_draw*100,prob_away*100)
    else:
        return(0,0)

In [61]:
fixture_group = fixture[:48].copy()
fixture_knockout = fixture[48:56].copy()
fixture_quarter = fixture[56:60].copy()
fixture_semi = fixture[60:62].copy()
fixture_final = fixture[62:].copy()

In [62]:
for group  in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    fixture_by_group = fixture_group[fixture_group['home'].isin(teams_in_group)]
    for index, row in fixture_by_group.iterrows():
        home,away = row['home'], row['away'] 
        points_home,points_away = predict_points(home,away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
    dict_table[group] = dict_table[group].sort_values('Pts', ascending = False).reset_index()
    dict_table[group] = dict_table[group][['Team','Pts']].round(0)

In [63]:
for group in dict_table:
    group_winner = dict_table[group].loc[0,'Team']
    group_runner_up = dict_table[group].loc[1,'Team']
    fixture_knockout.replace({f'Winners {group}':group_winner,f'Runners-up {group}':group_runner_up}, inplace=True)
fixture_knockout['Winner'] = '-'

def get_winner(fixture_updated):
    for index, row in fixture_updated.iterrows():
        points_home, points_away = predict_points(row['home'],row['away'])
        if points_home > points_away:
            Winner = row['home']
        else:
            Winner = row['away']
        fixture_updated.loc[index, 'Winner'] = Winner
    return fixture_updated

In [64]:
get_winner(fixture_knockout)

,home,score,away,year,Winner
48,Netherlands,Match 49,United States,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Mexico,2022,France
51,England,Match 51,Senegal,2022,England
52,Spain,Match 53,Croatia,2022,Spain
53,Brazil,Match 54,Portugal,2022,Brazil
54,Belgium,Match 55,Germany,2022,Germany
55,Uruguay,Match 56,Switzerland,2022,Switzerland


In [65]:
def update_table(fixture_1,fixture_2):
    for index, row in fixture_1.iterrows():
        winner = fixture_1.loc[index,'Winner']
        match = fixture_1.loc[index, 'score']
        fixture_2.replace({f'Winners {match}':winner},inplace=True)
    fixture_2['Winner']='-'
    return fixture_2


In [66]:
update_table(fixture_knockout,fixture_quarter)
get_winner(fixture_quarter)

,home,score,away,year,Winner
56,Spain,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Germany,Match 60,Switzerland,2022,Germany
59,England,Match 59,France,2022,France


In [67]:
update_table(fixture_quarter,fixture_semi)
get_winner(fixture_semi)

,home,score,away,year,Winner
60,Netherlands,Match 61,Brazil,2022,Netherlands
61,France,Match 62,Germany,2022,Germany


In [68]:
update_table(fixture_semi,fixture_final)
get_winner(fixture_final)

,home,score,away,year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Netherlands,Match 64,Germany,2022,Germany
